In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
import uuid

def create_embeddings(dataframe, text_column):
    """
    DataFrame의 특정 텍스트 컬럼을 임베딩하여 새로운 컬럼에 추가합니다.
    """
    try:
        # 한국어에 특화된 임베딩 모델을 로드합니다.
        model = SentenceTransformer('jhgan/ko-sroberta-multitask')
        sentences = dataframe[text_column].tolist()
        
        print("텍스트 임베딩을 시작합니다...")
        embeddings = model.encode(sentences, show_progress_bar=True)
        print("텍스트 임베딩 완료.")

        dataframe['embeddings'] = embeddings.tolist()
        return dataframe
    
    except Exception as e:
        print(f"임베딩 과정에서 오류가 발생했습니다: {e}")
        return None

def store_to_chroma(dataframe, db_path, collection_name):
    """
    임베딩된 DataFrame을 ChromaDB에 저장합니다.
    """
    try:
        # ChromaDB 클라이언트 초기화 (로컬 폴더에 데이터베이스 파일 생성)
        client = chromadb.PersistentClient(path=db_path)
        
        # 컬렉션(Collection) 생성 또는 불러오기
        # '컬렉션'은 데이터를 저장하는 공간으로, SQL의 '테이블'과 비슷합니다.
        collection = client.get_or_create_collection(name=collection_name)
        
        # DataFrame의 데이터를 ChromaDB에 추가
        # id는 고유해야 하므로 uuid를 사용
        ids = [str(uuid.uuid4()) for _ in range(len(dataframe))]
        documents = dataframe['title'].tolist()
        embeddings = dataframe['embeddings'].tolist()
        
        print(f"ChromaDB '{collection_name}' 컬렉션에 데이터를 저장 중입니다...")
        collection.add(
            embeddings=embeddings,
            documents=documents,
            ids=ids
        )
        print(f"ChromaDB에 {len(documents)}개의 문서가 성공적으로 저장되었습니다.")
        
    except Exception as e:
        print(f"ChromaDB 저장 중 오류가 발생했습니다: {e}")

# 실행 코드
if __name__ == "__main__":
    keyword = input("벡터 데이터베이스에 저장할 데이터의 키워드를 입력하세요: ").strip()
    filename = f"./data/{keyword}_naver_news.csv"

    try:
        # 1. 크롤링한 CSV 파일을 불러오기
        df = pd.read_csv(filename)
        
        # 2. 텍스트를 임베딩
        df_with_embeddings = create_embeddings(df, 'title')
        
        if df_with_embeddings is not None:
            print("\n임베딩된 데이터프레임의 첫 3개 행:")
            print(df_with_embeddings.head(3))
            
            # 3. 임베딩된 데이터를 ChromaDB에 저장
            db_path = f"./chroma_db_{keyword}"
            collection_name = f"{keyword}_news_collection"
            store_to_chroma(df_with_embeddings, db_path, collection_name)
            
    except FileNotFoundError:
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

벡터 데이터베이스에 저장할 데이터의 키워드를 입력하세요:  AI


텍스트 임베딩을 시작합니다...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

텍스트 임베딩 완료.

임베딩된 데이터프레임의 첫 3개 행:
   Unnamed: 0                                              title  \
0           0                    최교진 교육부 장관 취임…“공교육 회복 위해 교권 보호”   
1           1                  코레일, 추석 예매 앞두고 주요 철도역서 '디지털스쿨' 운영   
2           2  SK에코플랜트, <b>AI</b>·로봇·반도체 혁신기술 공모전 ‘2025 콘테크 미...   

                                        originallink  \
0  http://www.edaily.co.kr/news/newspath.asp?news...   
1  https://www.redaily.co.kr/news/articleView.htm...   
2  https://www.seoultimes.news/news/article.html?...   

                                                link  \
0  https://n.news.naver.com/mnews/article/018/000...   
1  https://www.redaily.co.kr/news/articleView.htm...   
2  https://www.seoultimes.news/news/article.html?...   

                                         description  \
0  인공지능(<b>AI</b>) 인재를 적극 육성하겠다는 의지도 내비쳤다. 이를 위해 ...   
1  코레일 직원들은 역 맞이방에서 스마트폰이 익숙치 않은 어르신 등에게 △모바일 앱 '...   
2  <b>AI</b>·로봇·반도체 혁신기술 보유 스타트업 및 중소·중견기업 대상 오픈 ...   

                  